In [1]:
# repo imports
from main import run_backtest
from risk_manager import ExecParams
import logging, os


/Users/tammiekoh/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Briefly describe: MA + Trailing Stop, cooldown, min-hold, weekly exec, ATR catastrophic, giveback, time stop, vol targeting, gross-cap, rolling-CAGR filter.

In [2]:
# backtest parameters (feel free to adjust)
MA_WINDOW = 130
TRAIL_WINDOW = 10
TRAIL_PCT = 0.93
SHORT_THRESHOLD = 0.02
ATR_WIN = 14

COOLDOWN = 3
EPS = 0.005
MIN_HOLD = 5
WEEKLY = True
K_ATR = 2.5
GIVEBACK = 2.0
TIME_STOP = 60

TARGET_VOL = 0.10
GROSS_CAP = 1.0
ROLL_VOL = 20
PERF_LOOKBACK = 63

In [3]:
start_date = "2015-01-01"
end_date   = "2019-12-31"

# stock_list = ["NVDA","MSFT","AAPL","AMZN","META","AVGO","GOOGL","TSLA","GOOG","BRK-B"]
stock_list = ["META","AAPL","AMZN","NFLX","GOOG"]

In [4]:
results = run_backtest(
    start_date=start_date, end_date=end_date, stock_list=stock_list,
    ma_window=MA_WINDOW, trailing_window=TRAIL_WINDOW, trailing_pct=TRAIL_PCT,
    short_threshold=SHORT_THRESHOLD, atr_win=ATR_WIN,
    cooldown_bars=COOLDOWN, reentry_eps=EPS, min_hold=MIN_HOLD, weekly_exec=WEEKLY,
    k_atr_stop=K_ATR, giveback_atr=GIVEBACK, time_stop=TIME_STOP,
    target_vol=TARGET_VOL, gross_cap=GROSS_CAP, roll_vol=ROLL_VOL, perf_lookback=PERF_LOOKBACK,
    initial_capital=500_000.0, compare_spy=True, tag="final"
)
results["portfolio_metrics"]

[NOTIFY] Loading 5 symbols: META, AAPL, AMZN, NFLX, GOOG
[NOTIFY] META: 2015-01-02 → 2019-12-30 (1257 rows)
[NOTIFY] AAPL: 2015-01-02 → 2019-12-30 (1257 rows)
[NOTIFY] AMZN: 2015-01-02 → 2019-12-30 (1257 rows)
[NOTIFY] NFLX: 2015-01-02 → 2019-12-30 (1257 rows)
[NOTIFY] GOOG: 2015-01-02 → 2019-12-30 (1257 rows)
[NOTIFY] Saved returns → /Users/tammiekoh/Documents/is4226-final/storage/returns.csv
[NOTIFY] Saved trades  → /Users/tammiekoh/Documents/is4226-final/storage/trades.csv


{'CAGR': 0.2564373029292353,
 'Sharpe': 2.390901987424725,
 'MaxDrawdown': -0.09976788855027396,
 'Calmar': 2.5703390805951973}

In [5]:
log_dir = "./storage/logs"
os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    filename=os.path.join(log_dir, "run_backtest.log"),
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

logging.info(
    f"Backtest completed | Period: {start_date} → {end_date} | "
    f"Params: MA={MA_WINDOW}, Trail={TRAIL_WINDOW}, Pct={TRAIL_PCT}, "
    f"ShortThr={SHORT_THRESHOLD}, ATR={ATR_WIN} | "
    f"Metrics: {results['portfolio_metrics']}"
)

print("✅ Backtest metrics logged to storage/logs/run_backtest.log")

✅ Backtest metrics logged to storage/logs/run_backtest.log
